ADAPTIVE PERSUASION SYSTEM - PRODUCTION VERSION

In [362]:
!pip install -q transformers>=4.40.0 huggingface_hub>=0.23.0 torch
!pip install -q textblob nltk pandas numpy matplotlib

import torch, transformers, textblob, nltk, pandas, numpy, matplotlib
print("All required libraries are installed and imported successfully.")


All required libraries are installed and imported successfully.


In [363]:
# Imports

import os
import json
import re
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import random

from huggingface_hub import InferenceClient, login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from textblob import TextBlob
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML


# NLTK Downloads
nltk.download('punkt', quiet=True)
nltk.download('vader_lexicon', quiet=True)


True

In [364]:
# Authentication

HF_TOKEN = os.getenv("HF_TOKEN")
os.environ['HF_TOKEN'] = HF_TOKEN

try:
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("Authenticated\n")
except Exception as e:
    print(f"Failed: {e}")
    raise

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Authenticated



In [365]:
# Loading the llama 3.1 model

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

'''
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        token=HF_TOKEN,
        dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    print("Model loaded successfully!\n")
    USE_LOCAL_MODEL = True
except Exception as e:
    print(f"Could not load model locally: {e}")
    print("Falling back to Inference API until access is approved\n")
    USE_LOCAL_MODEL = False
    client = InferenceClient(api_key=HF_TOKEN)
'''

print("Using Hugging Face Inference API (API-only mode) for now\n")

USE_LOCAL_MODEL = False
client = InferenceClient(api_key=HF_TOKEN)



Using Hugging Face Inference API (API-only mode) for now



In [366]:
# CAMPAIGN SETUP

org_name = input("Organization [Children's Education Fund]: ").strip() or "Children's Education Fund"
cause = input("Cause [education for children]: ").strip() or "providing education to underprivileged children"
amounts = input("Amounts [200,500,1000]: ").strip() or "200, 500, 1000"
impact = input("Impact [₹200 = 5 kids supplies]: ").strip() or "₹200 provides school supplies for 5 children for a month"

DONATION_CONTEXT = {
    'organization': org_name,
    'cause': cause,
    'amounts': amounts,
    'impact': impact
}

print("\nCampaign ready\n")


Campaign ready



In [367]:
'''
# CONFIGURATION 1 for dev mode

class Config:
    MODEL_NAME = MODEL_NAME
    TEMPERATURE = 0.8
    MAX_NEW_TOKENS = 64

    INITIAL_BELIEF = 0.05
    INITIAL_TRUST = 1.0
    TRUST_THRESHOLD = 0.5

    ALPHA = 0.1
    BETA = 0.15
    GAMMA = 0.08

    HARD_REJECTION_PENALTY = 0.4
    SOFT_REJECTION_PENALTY = 0.2
    MIN_STRATEGY_WEIGHT = 0.05

    MAX_TURNS = 15
    MAX_CONSECUTIVE_REJECTIONS = 3

    LOG_FILE = "dialogue_log.jsonl"
    STRATEGIES = ["Empathy", "Impact", "SocialProof", "Transparency", "EthicalUrgency"]
'''

# CONFIGURATION 2 for demo mode (more visual changes with the same logic)

class Config:
    MODEL_NAME = MODEL_NAME
    TEMPERATURE = 0.8
    MAX_NEW_TOKENS = 64

    # ---- Initial states ----
    INITIAL_BELIEF = 0.15    # start higher so drops are visible
    INITIAL_TRUST = 0.9     # not perfect trust at start
    TRUST_THRESHOLD = 0.5

    # ---- Learning rates (KEY CHANGE) ----
    ALPHA = 0.35   # belief moves clearly each turn
    BETA = 0.4     # trust reacts noticeably to skepticism
    GAMMA = 0.15   # recovery is visible but not instant

    # ---- Strategy adaptation ----
    HARD_REJECTION_PENALTY = 0.6
    SOFT_REJECTION_PENALTY = 0.35
    MIN_STRATEGY_WEIGHT = 0.05

    # ---- Conversation limits ----
    MAX_TURNS = 15
    MAX_CONSECUTIVE_REJECTIONS = 3

    LOG_FILE = "dialogue_log.jsonl"

    STRATEGIES = [
        "Empathy",
        "Impact",
        "SocialProof",
        "Transparency",
        "EthicalUrgency"
    ]

In [368]:
# REJECTION DETECTOR

class RejectionDetector:

    EXPLICIT_PATTERNS = [
        r'\b(no thanks|no thank you|not interested|don\'t want|won\'t donate)\b',
        r'\b(nope|nah|never|absolutely not|definitely not)\b',
        r'\b(leave me alone|stop asking|not doing|refuse)\b'
    ]

    POLITE_EXIT_PATTERNS = [
        r'\b(okay thanks|ok thanks|okay thank you|ok thank you)\b',
        r'\b(thank you|thanks|appreciate it)\b$',
        r'\b(that\'s all|that is all|nothing else)\b',
        r'\b(i\'m good|all good|we\'re good)\b'
    ]

    SOFT_PATTERNS = [
        r'\b(maybe later|not now|not right now|some other time|another time)\b',
        r'\b(not today|can\'t right now|busy right now)\b',
        r'\b(not sure|unsure|uncertain|hesitant|doubtful)\b',
        r'\b(can\'t afford|no money|tight budget|broke|expensive)\b',
        r'\b(i\'ll think|let me think|need time|consider)\b'
    ]

    TRUST_PATTERNS = [
        r'\b(pushy|aggressive|pressure|uncomfortable|sketchy|scam|fraud)\b',
        r'\b(suspicious|don\'t trust|seems fake|sounds fake)\b',
        r'\b(why are you|what\'s your motive|prove it)\b'
    ]

    CURIOSITY_PATTERNS = [
        r'\b(tell me more|tell me about|what about|explain|how does|how do)\b',
        r'\b(more info|more details|details|information)\b',
        r'\b(curious|interested in learning|want to know|want to hear)\b',
        r'\b(what is|who are|where does|when|why)\b',
        r'\b(can you|could you|would you.*explain|show me)\b'
    ]

    ACCEPTANCE_PATTERNS = [
        r'\b(yes.*i.*donate|i will donate|i\'ll donate|i want to donate)\b',
        r'\b(sign me up|count me in|i\'m in)\b',
        r'\b(where do i donate|how do i donate|how can i donate)\b',
        r'\b(i\'ll give|i will give|i want to give ₹|i can give ₹)\b',
        r'\b(okay.*donate|ok.*donate|sure.*donate|let\'s do it)\b',
        r'\b(take my donation|here\'s my donation|ready to donate)\b'
    ]

    def __init__(self):
        self.sentiment = None

    def detect(self, user_message: str) -> Dict:
        msg = user_message.lower().strip()

        # ---- Polite exit (NOT a refusal) ----
        is_polite_exit = self._match(msg, self.POLITE_EXIT_PATTERNS)

        # ---- Acceptance ----
        if self._match(msg, self.ACCEPTANCE_PATTERNS):
            return {
                'rejection_type': 'none',
                'rejection_confidence': 0.0,
                'trust_concern': False,
                'sentiment_score': 0.9,
                'sentiment_label': 'positive',
                'is_acceptance': True,
                'is_curiosity': False,
                'is_polite_exit': False
            }

        # Curiosity
        is_curiosity = self._match(msg, self.CURIOSITY_PATTERNS)

        # Rejection
        rejection_type = 'none'
        confidence = 0.0

        if self._match(msg, self.EXPLICIT_PATTERNS):
            rejection_type = 'explicit'
            confidence = 0.9
        elif self._match(msg, self.SOFT_PATTERNS):
            rejection_type = 'soft'
            confidence = 0.7

        trust_concern = self._match(msg, self.TRUST_PATTERNS)
        sent_score, sent_label = self._get_sentiment(user_message)

        if is_curiosity and rejection_type == 'none':
            rejection_type = 'curiosity'
            sent_score = max(0.2, sent_score)

        if sent_score < -0.4 and rejection_type == 'none':
            rejection_type = 'ambiguous'
            confidence = 0.5

        if sent_score < -0.6 and rejection_type == 'soft':
            rejection_type = 'explicit'
            confidence = 0.85

        return {
            'rejection_type': rejection_type,
            'rejection_confidence': confidence,
            'trust_concern': trust_concern,
            'sentiment_score': sent_score,
            'sentiment_label': sent_label,
            'is_acceptance': False,
            'is_curiosity': is_curiosity,
            'is_polite_exit': is_polite_exit
        }

    def _match(self, text: str, patterns: List[str]) -> bool:
        return any(re.search(p, text, re.IGNORECASE) for p in patterns)

    def _get_sentiment(self, text: str) -> Tuple[float, str]:
        blob = TextBlob(text)
        pol = blob.sentiment.polarity
        label = 'positive' if pol > 0 else ('negative' if pol < 0 else 'neutral')
        return pol, label


In [369]:
# BELIEF & TRUST TRACKERS

class BeliefTracker:
    def __init__(self):
        self.belief = Config.INITIAL_BELIEF
        self.history = [self.belief]

    def update(self, rejection_info: Dict, trust: float) -> float:
        rtype = rejection_info['rejection_type']
        is_accept = rejection_info['is_acceptance']
        is_curious = rejection_info['is_curiosity']
        trust_concern = rejection_info['trust_concern']
        sent = rejection_info['sentiment_score']

        if is_accept:
            effect = (1 - self.belief) * 0.9
        elif rtype == 'explicit':
            effect = -0.9
        elif trust_concern:
            effect = -0.7
        elif rtype == 'soft':
            effect = -0.45
        elif rtype == 'ambiguous':
            effect = -0.25
        elif is_curious:
            effect = 0.25
        elif sent > 0.3:
            effect = 0.15
        else:
            effect = 0.0

        delta = Config.ALPHA * effect

        # Trust gating
        if trust < Config.TRUST_THRESHOLD and delta > 0:
            delta = 0.0

        self.belief = np.clip(self.belief + delta, 0, 1)
        self.history.append(self.belief)
        return delta

    def get(self) -> float:
        return self.belief

class TrustTracker:
    def __init__(self):
        self.trust = Config.INITIAL_TRUST
        self.history = [self.trust]
        self.recovery_mode = False

    def update(self, rejection_info: Dict, strategy: str):
        rtype = rejection_info['rejection_type']
        concern = rejection_info['trust_concern']

        delta = 0.0

        # Trust erosion
        if concern:
            delta = -Config.BETA * 0.6
        elif rtype in ['soft', 'ambiguous']:
            delta = -Config.BETA * 0.3
        elif rtype == 'explicit':
            delta = -Config.BETA * 0.5

        # Trust recovery
        elif strategy == 'Transparency':
            delta = Config.GAMMA
        elif rejection_info['is_curiosity']:
            delta = Config.GAMMA * 0.3

        self.trust = np.clip(self.trust + delta, 0, 1)
        self.history.append(self.trust)

        # Recovery mode logic
        if self.trust < Config.TRUST_THRESHOLD:
            self.recovery_mode = True
        elif self.trust >= Config.TRUST_THRESHOLD:
            self.recovery_mode = False

        return delta, self.recovery_mode

    def get(self):
        return self.trust


In [370]:
# STRATEGY ADAPTER

class StrategyAdapter:
    # Hard allowlist for recovery mode (PDF-aligned)
    RECOVERY_ALLOWED = {'Empathy', 'Transparency'}

    def __init__(self):
        n = len(Config.STRATEGIES)
        self.weights = {s: 1.0/n for s in Config.STRATEGIES}
        self.history = {s: [1.0/n] for s in Config.STRATEGIES}
        self.count = {s: 0 for s in Config.STRATEGIES}

    def select(self, in_recovery: bool) -> str:
        # -------- HARD TRUST CONSTRAINT --------
        if in_recovery:
            available = {
                s: self.weights[s]
                for s in self.RECOVERY_ALLOWED
                if s in self.weights
            }
        else:
            available = self.weights.copy()
        # --------------------------------------

        strats = list(available.keys())
        wts = list(available.values())

        # Safety fallback (should never happen, but defensive)
        if not strats:
            strats = ['Empathy']
            wts = [1.0]

        total = sum(wts)
        if total == 0:
            wts = [1.0/len(strats)] * len(strats)
        else:
            wts = [w/total for w in wts]

        chosen = np.random.choice(strats, p=wts)
        self.count[chosen] += 1
        return chosen

    def adapt(self, strategy: str, rejection_info: Dict):
        # IMPORTANT: Do not adapt forbidden strategies in recovery
        # (weights are preserved but not used)
        rtype = rejection_info['rejection_type']
        is_accept = rejection_info['is_acceptance']
        is_curious = rejection_info['is_curiosity']

        if is_accept:
            self.weights[strategy] = min(1.0, self.weights[strategy] * 1.5)
        elif is_curious:
            self.weights[strategy] = min(1.0, self.weights[strategy] * 1.2)
        elif rtype == 'explicit':
            self.weights[strategy] = max(
                Config.MIN_STRATEGY_WEIGHT,
                self.weights[strategy] * (1 - Config.HARD_REJECTION_PENALTY)
            )
        elif rtype == 'soft':
            self.weights[strategy] = max(
                Config.MIN_STRATEGY_WEIGHT,
                self.weights[strategy] * (1 - Config.SOFT_REJECTION_PENALTY)
            )

        if rejection_info['trust_concern']:
            self.weights[strategy] = max(
                Config.MIN_STRATEGY_WEIGHT,
                self.weights[strategy] * 0.7
            )
            self.weights['Transparency'] = min(
                1.0,
                self.weights['Transparency'] * 1.3
            )

        # Renormalize
        total = sum(self.weights.values())
        if total > 0:
            self.weights = {s: w/total for s, w in self.weights.items()}

        for s in Config.STRATEGIES:
            self.history[s].append(self.weights[s])


In [371]:
# GUARDRAILS

class Guardrails:
    def __init__(self):
        self.turn = 0
        self.consec_reject = 0   # real signal for disengagement

    def check(
        self,
        rejection_info: Dict,
        trust: float,
        belief: float
    ) -> Tuple[bool, Optional[str]]:
        self.turn += 1

        rtype = rejection_info['rejection_type']
        is_polite_exit = rejection_info.get('is_polite_exit', False)

        # ---- Track consecutive rejections ----
        if rtype in ['explicit', 'soft', 'ambiguous']:
            self.consec_reject += 1
        else:
            self.consec_reject = 0

        # ---- DEBUG ----
        print(
            f"[GUARD DEBUG] "
            f"turn={self.turn} | "
            f"rtype={rtype} | "
            f"polite_exit={is_polite_exit} | "
            f"consec_reject={self.consec_reject} | "
            f"trust={trust:.2f} | "
            f"belief={belief:.2f}"
        )
        # ---------------

        # ---- ACCEPTANCE ----
        if rejection_info['is_acceptance']:
            return True, "User accepted"

        # ---- HARD EXIT: real explicit refusal ----
        if rtype == 'explicit' and not is_polite_exit:
            return True, "User declined donation"

        # ---- POLITE EXIT (ONLY AFTER RESISTANCE) ----
        # This is the key fix
        if is_polite_exit and self.consec_reject >= 1:
            return True, "User ended conversation"

        # ---- SAFETY EXITS ----
        if self.turn >= Config.MAX_TURNS:
            return True, f"Max turns ({Config.MAX_TURNS})"

        if trust < 0.3:
            return True, "Trust too low"

        return False, None


In [372]:
# LLM AGENT

class LLMAgent:
    def __init__(self, donation_ctx: Dict):
        self.ctx = donation_ctx
        self.conversation_memory = []

    def generate(self, strategy: str, user_msg: str, turn: int,
                is_recovery: bool, sentiment: str) -> str:

        # Build conversation context
        recent_history = self.conversation_memory[-3:] if len(self.conversation_memory) > 0 else []
        history_str = ""
        for h in recent_history:
            history_str += f"User: {h['user']}\nAgent: {h['agent']}\n"

        # Build prompt
        if is_recovery:
            prompt = self._recovery_prompt(user_msg, history_str, sentiment)
        else:
            prompt = self._strategy_prompt(strategy, user_msg, history_str, turn, sentiment)

        # Generate
        try:
            if USE_LOCAL_MODEL:
                response = self._generate_local(prompt)
            else:
                response = self._generate_api(prompt)
        except Exception as e:
            print(f"Generation error: {e}")
            response = self._fallback(strategy, is_recovery)

        # Store in memory
        self.conversation_memory.append({
            'user': user_msg,
            'agent': response
        })

        return response

    def _strategy_prompt(self, strategy: str, user_msg: str, history: str,
                        turn: int, sentiment: str) -> str:

        strategy_guides = {
            "Empathy": "Respond with empathy and understanding. Acknowledge their feelings warmly.",
            "Impact": f"Share concrete impact: {self.ctx['impact']}. Use numbers and specific outcomes.",
            "SocialProof": "Mention that others in the community are contributing. Make it aspirational.",
            "Transparency": "Be completely honest. Explain where money goes. Build trust through openness.",
            "EthicalUrgency": "Mention time-sensitive need gently. No pressure. Use soft phrases."
        }

        prompt = f"""You are a fundraising assistant for {self.ctx['organization']}, working on {self.ctx['cause']}.

Suggested donation amounts: ₹{self.ctx['amounts']}
Impact example: {self.ctx['impact']}

CONVERSATION SO FAR:
{history}

USER JUST SAID: "{user_msg}"
User seems: {sentiment}

YOUR STRATEGY: {strategy}
{strategy_guides.get(strategy, '')}

CRITICAL RULES:
- If they're asking questions, ANSWER them specifically
- Don't assume they want to donate from curiosity
- Keep under 50 words
- Be natural and conversational
- Build on previous conversation

Your response:"""

        return prompt

    def _recovery_prompt(self, user_msg: str, history: str, sentiment: str) -> str:
        return f"""You are a fundraising assistant for {self.ctx['organization']} in TRUST RECOVERY mode.

CONVERSATION SO FAR:
{history}

USER JUST SAID: "{user_msg}"
User seems: {sentiment}

They're uncomfortable. Your ONLY job:
1. Apologize sincerely
2. Reassure NO pressure
3. Offer to answer questions
4. Step back from donation completely

Keep under 40 words. Rebuild trust, NOT donation.

Your response:"""

    def _generate_local(self, prompt: str) -> str:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=Config.MAX_NEW_TOKENS,
                temperature=Config.TEMPERATURE,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )

        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = full_text[len(prompt):].strip()

        # Clean up
        response = response.split('\n')[0].strip()
        if len(response) > 300:
            response = response[:300].rsplit('.', 1)[0] + '.'

        return response

    def _generate_api(self, prompt: str) -> str:
        response = client.chat.completions.create(
            model=Config.MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a helpful, polite fundraising assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=Config.MAX_NEW_TOKENS,
            temperature=Config.TEMPERATURE,
        )
        return response.choices[0].message.content.strip()


    def _fallback(self, strategy: str, is_recovery: bool) -> str:
        if is_recovery:
            return "I apologize if I seemed pushy. There's no pressure at all - I'm happy to answer any questions you have."

        fallbacks = {
            "Empathy": "I understand where you're coming from. What questions do you have about our work?",
            "Impact": f"For context: {self.ctx['impact']}. Every contribution helps real families.",
            "SocialProof": "Many people in our community are supporting this cause. Would you like to learn more?",
            "Transparency": "I'm happy to share exactly where donations go and how they're used. What would you like to know?",
            "EthicalUrgency": "This month we're focused on urgent needs, but there's no pressure. What questions can I answer?"
        }
        return fallbacks.get(strategy, "Thank you for your time. What would you like to know?")


In [373]:
# DIALOGUE MANAGER

class DialogueManager:
    def __init__(self, condition: str, donation_ctx: Dict):
        self.condition = condition
        self.session_id = f"sess_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.ctx = donation_ctx

        self.agent = LLMAgent(donation_ctx)
        self.detector = RejectionDetector()
        self.belief = BeliefTracker()
        self.trust = TrustTracker()
        self.strategy = StrategyAdapter()
        self.guard = Guardrails()

        self.history = []
        self.turn = 0
        self.active = True
        self.outcome = None

        if condition == 'C1':
            self.static_strat = 'Empathy'

    def start(self) -> str:
        opening = (
            f"Hello! I'm from {self.ctx['organization']}. "
            f"We're working on {self.ctx['cause']}. "
            f"Would you like to learn more about what we do?"
        )
        self.history.append(
            {'turn': 0, 'speaker': 'agent', 'msg': opening, 'strategy': 'Empathy'}
        )
        return opening

    def process(self, user_msg: str) -> Dict:
        self.turn += 1

        # ---- Analyze ----
        rej_info = self.detector.detect(user_msg)

        # ---- Update belief FIRST (needs trust) ----
        prev_strat = self.history[-1]['strategy']
        delta_p = self.belief.update(rej_info, self.trust.get())

        # ---- Update trust ----
        delta_t = 0.0
        if self.condition == 'C3':
            delta_t, _ = self.trust.update(rej_info, prev_strat)

        # ---- Guardrails ----
        should_stop, reason = self.guard.check(
            rej_info,
            self.trust.get(),
            self.belief.get()
        )

        if should_stop:
            self.active = False
            self.outcome = reason
            return {
                'agent_msg': self._closing(reason),
                'metrics': self._metrics(rej_info, delta_p, delta_t),
                'stop': True,
                'reason': reason
            }

        # ---- Strategy selection ----
        if self.condition == 'C1':
            chosen = self.static_strat
        else:
            in_recovery = self.condition == 'C3' and self.trust.recovery_mode
            chosen = self.strategy.select(in_recovery)

        # ---- Adapt strategy ----
        if self.condition in ['C2', 'C3']:
            self.strategy.adapt(prev_strat, rej_info)


        # ---- Generate response ----
        agent_resp = self.agent.generate(
            chosen,
            user_msg,
            self.turn,
            self.trust.recovery_mode,
            rej_info['sentiment_label']
        )

        # ---- Log ----
        self.history.append(
            {'turn': self.turn, 'speaker': 'user', 'msg': user_msg, 'info': rej_info}
        )
        self.history.append(
            {'turn': self.turn, 'speaker': 'agent', 'msg': agent_resp, 'strategy': chosen}
        )

        return {
            'agent_msg': agent_resp,
            'metrics': self._metrics(rej_info, delta_p, delta_t),
            'stop': False,
            'reason': None
        }

    def _metrics(self, rej_info: Dict, dp: float, dt: float) -> Dict:
        return {
            'turn': self.turn,
            'belief': round(self.belief.get(), 3),
            'trust': round(self.trust.get(), 3),
            'delta_belief': round(dp, 3),
            'delta_trust': round(dt, 3),

            'rejection_type': rej_info['rejection_type'],
            'rejection_conf': round(rej_info.get('rejection_confidence', 0.0), 3),

            'sentiment': rej_info['sentiment_label'],
            'sentiment_score': round(rej_info['sentiment_score'], 3),
            'trust_concern': rej_info['trust_concern'],
            'is_curiosity': rej_info['is_curiosity'],

            'recovery_mode': self.trust.recovery_mode,
            'strategy_weights': {
                k: round(v, 3) for k, v in self.strategy.weights.items()
            },
            'consec_reject': self.guard.consec_reject
        }


    def _closing(self, reason: str) -> str:
        if 'accepted' in reason.lower():
            return "Thank you so much! Your donation will make a real difference."
        else:
            return "Thank you for your time. I respect your decision."

    def save(self):
        log = {
            'session_id': self.session_id,
            'condition': self.condition,
            'timestamp': datetime.now().isoformat(),
            'context': self.ctx,
            'history': self.history,
            'final_belief': self.belief.get(),
            'final_trust': self.trust.get(),
            'turns': self.turn,
            'outcome': self.outcome
        }
        with open(Config.LOG_FILE, 'a') as f:
            f.write(json.dumps(log) + '\n')


In [374]:
# =============================================================================
# VISUALIZATION (DARK MODE SAFE)
# =============================================================================

class Viz:
    @staticmethod
    def show_metrics(m: Dict):
        bc = '#4CAF50' if m['belief'] > 0.6 else ('#FFC107' if m['belief'] > 0.3 else '#F44336')
        tc = '#4CAF50' if m['trust'] > 0.7 else ('#FFC107' if m['trust'] > 0.5 else '#F44336')
        rc = '#F44336' if m['rejection_type'] == 'explicit' else (
             '#FFC107' if m['rejection_type'] == 'soft' else '#4CAF50')

        html = f"""
<div style="
    background:#1e1e1e;
    color:#e6e6e6;
    padding:16px;
    border-radius:10px;
    margin:12px 0;
    font-family: Arial, sans-serif;
">
    <h3 style="color:#ffffff; margin-bottom:10px;">
        Turn {m['turn']} Metrics
    </h3>

    <table style="width:100%;">
        <tr>
            <td style="width:50%; vertical-align:top; padding-right:15px;">
                <h4 style="color:#dddddd;">Belief & Trust</h4>
                <p>
                    <b>Donation Probability:</b>
                    <span style="color:{bc}; font-weight:bold;">
                        {m['belief']:.1%}
                    </span>
                    ({m['delta_belief']:+.3f})
                </p>
                <p>
                    <b>Trust Score:</b>
                    <span style="color:{tc}; font-weight:bold;">
                        {m['trust']:.1%}
                    </span>
                    ({m['delta_trust']:+.3f})
                </p>
                {f'<p style="color:#F44336; font-weight:bold;">RECOVERY MODE</p>' if m['recovery_mode'] else ''}
            </td>

            <td style="width:50%; vertical-align:top;">
                <h4 style="color:#dddddd;">Analysis</h4>
                <p>
                    <b>Rejection:</b>
                    <span style="color:{rc}; font-weight:bold;">
                        {m['rejection_type'].upper()}
                    </span>
                    ({m['rejection_conf']:.0%})
                </p>
                <p><b>Sentiment:</b> {m['sentiment']} ({m['sentiment_score']:+.2f})</p>
                <p><b>Curiosity:</b> {'Yes' if m['is_curiosity'] else 'No'}</p>
                <p><b>Trust Issue:</b> {'YES' if m['trust_concern'] else 'No'}</p>
            </td>
        </tr>
    </table>

    <h4 style="color:#dddddd; margin-top:12px;">Strategy Weights</h4>
    <div style="font-size:0.9em;">
"""

        for s, w in sorted(m['strategy_weights'].items(), key=lambda x: x[1], reverse=True):
            bar = int(w * 180)
            html += f"""
        <div style="margin:6px 0;">
            <span style="display:inline-block; width:120px; color:#cccccc;">
                {s}:
            </span>
            <div style="
                display:inline-block;
                width:180px;
                background:#333333;
                border-radius:4px;
                vertical-align:middle;
            ">
                <div style="
                    width:{bar}px;
                    background:#4CAF50;
                    height:12px;
                    border-radius:4px;
                "></div>
            </div>
            <span style="margin-left:8px; color:#cccccc;">
                {w:.1%}
            </span>
        </div>
"""

        html += """
    </div>
</div>
"""
        display(HTML(html))

    @staticmethod
    def plot(dm: DialogueManager):
        bh = dm.belief.history
        th = dm.trust.history

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        ax1.plot(bh, marker='o', linewidth=2, markersize=6, label='Belief')
        ax1.set_xlabel('Turn')
        ax1.set_ylabel('P(donate)')
        ax1.set_title('Donation Probability Over Time')
        ax1.set_ylim([0, 1])
        ax1.grid(True, alpha=0.3)
        ax1.legend()

        ax2.plot(th, marker='s', linewidth=2, markersize=6, label='Trust')
        ax2.axhline(
            y=Config.TRUST_THRESHOLD,
            linestyle='--',
            label=f'Recovery Threshold ({Config.TRUST_THRESHOLD})'
        )
        ax2.set_xlabel('Turn')
        ax2.set_ylabel('Trust Score')
        ax2.set_title('Trust Score Over Time')
        ax2.set_ylim([0, 1])
        ax2.grid(True, alpha=0.3)
        ax2.legend()

        plt.tight_layout()
        plt.show()


In [375]:
# MAIN CONVERSATION LOOP

def run_conversation(condition: str = 'C3'):
    if condition in ['1', '2']:
        condition = 'C1' if condition == '1' else 'C3'
    print("\n" + "="*80)
    print("ADAPTIVE PERSUATION SYSTEM")
    print("="*80)

    # Updated condition labels: C1 renamed, C2 removed, C3 renamed
    desc = {
        'C1': "REGULAR CHATBOT",
        'C3': "ADAPTIVE PERSUATION SYSTEM"
    }

    print(f"\nCondition: {condition} - {desc.get(condition, 'Unknown')}")
    print("\nYOU are the USER - AI will try to persuade you to donate")
    print("Chat naturally - ask questions, say no, express concerns")
    print("Metrics shown after EACH response")
    print("\nType 'quit' to end\n")
    print("="*80 + "\n")

    dm = DialogueManager(condition, DONATION_CONTEXT)

    opening = dm.start()
    print(f"AGENT: {opening}\n")

    while dm.active:
        user_input = input("YOU: ").strip()

        if not user_input:
            print("Please enter something\n")
            continue

        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("\nEnding early...")
            dm.outcome = "User quit"
            break

        print("\nProcessing...\n")

        # ---- MAIN STEP ----
        result = dm.process(user_input)
        # -------------------

        print(f"AGENT: {result['agent_msg']}\n")

        print("─" * 80)
        Viz.show_metrics(result['metrics'])
        print("─" * 80 + "\n")

        if result['stop']:
            print(f"\nEnded: {result['reason']}\n")
            break

    print("\n" + "="*80)
    print("FINAL SUMMARY")
    print("="*80)
    print(f"Turns: {dm.turn}")
    print(f"Final Belief: {dm.belief.get():.1%}")
    print(f"Final Trust: {dm.trust.get():.1%}")
    print(f"Outcome: {dm.outcome}")
    print("="*80 + "\n")

    print("Generating plots...\n")
    Viz.plot(dm)

    dm.save()

    return dm


In [ ]:
# EXECUTION

if __name__ == "__main__":
    print("\n" + "="*80)
    print("SELECT CONDITION")
    print("="*80)
    print("1 - REGULAR CHATBOT")
    print("2 - ADAPTIVE PERSUATION SYSTEM")
    print("\nChoose one of the conditions: (Enter '1' or '2')")
    print("="*80)

    cond = input("\nCondition : ").strip().upper() or '2'

    if cond == '1':
        cond = '1'
    else:
        cond = '2'

    if cond not in ['1', '2']:
        print(f"Invalid, using default Adaptive Persuasion System (2)")
        cond = '2'

    dm = run_conversation(cond)

    print("\nCOMPLETE!")
    print(f"Log saved: {Config.LOG_FILE}")
    print("\nRun again: run_conversation")


SELECT CONDITION
1 - REGULAR CHATBOT
2 - ADAPTIVE PERSUATION SYSTEM

Choose one of the conditions: (Enter '1' or '2')

ADAPTIVE PERSUATION SYSTEM

Condition: C3 - ADAPTIVE PERSUATION SYSTEM

YOU are the USER - AI will try to persuade you to donate
Chat naturally - ask questions, say no, express concerns
Metrics shown after EACH response

Type 'quit' to end


AGENT: Hello! I'm from Children's Education Fund. We're working on providing education to underprivileged children. Would you like to learn more about what we do?


Processing...

[GUARD DEBUG] turn=1 | rtype=soft | polite_exit=False | consec_reject=1 | trust=0.78 | belief=0.00
AGENT: That's completely understandable. I just wanted to share a quick update on how donations like these can make a difference. For instance, a donation of just ₹200 can provide school supplies for 5 children for an entire month. That's a tangible impact for kids in need.

────────────────────────────────────────────────────────────────────────────────


Belief & Trust Donation Probability: 0.0% (-0.158) Trust Score: 78.0% (-0.120),Analysis Rejection: SOFT (70%) Sentiment: neutral (+0.00) Curiosity: No Trust Issue: No


────────────────────────────────────────────────────────────────────────────────


Processing...

[GUARD DEBUG] turn=2 | rtype=soft | polite_exit=False | consec_reject=2 | trust=0.66 | belief=0.00
AGENT: No worries at all. Maybe "later" is just a good time to plan ahead. September is just around the corner, and we're gearing up for the new school season. Your support now can make a big difference for kids who are waiting for school supplies.

────────────────────────────────────────────────────────────────────────────────


Belief & Trust Donation Probability: 0.0% (-0.158) Trust Score: 66.0% (-0.120),Analysis Rejection: SOFT (70%) Sentiment: neutral (+0.00) Curiosity: No Trust Issue: No


────────────────────────────────────────────────────────────────────────────────


Processing...

[GUARD DEBUG] turn=3 | rtype=curiosity | polite_exit=False | consec_reject=0 | trust=0.71 | belief=0.09
AGENT: I completely understand your concern. At Children's Education Fund, we're 100% transparent about how donations are used. Your ₹200 would go directly towards buying school supplies, and we'll share photos of the kids receiving them on our website. No middlemen, no hidden costs.

────────────────────────────────────────────────────────────────────────────────


...

────────────────────────────────────────────────────────────────────────────────


Processing...

[GUARD DEBUG] turn=4 | rtype=none | polite_exit=False | consec_reject=0 | trust=0.47 | belief=0.00
AGENT: I completely understand your skepticism, and I apologize if our previous conversations made you uncomfortable. There's no pressure at all; I'm here to provide information and answer any questions you may have.

────────────────────────────────────────────────────────────────────────────────


Belief & Trust Donation Probability: 0.0% (-0.245) Trust Score: 46.5% (-0.240) RECOVERY MODE,Analysis Rejection: NONE (0%) Sentiment: neutral (+0.00) Curiosity: No Trust Issue: YES


────────────────────────────────────────────────────────────────────────────────


Processing...

[GUARD DEBUG] turn=5 | rtype=none | polite_exit=False | consec_reject=0 | trust=0.47 | belief=0.00
AGENT: I completely understand and apologize if I made you uncomfortable. There's absolutely no pressure at all. I'm here to answer any questions you have. Let's start fresh with a conversation, no fundraising agenda.

────────────────────────────────────────────────────────────────────────────────


Belief & Trust Donation Probability: 0.0% (+0.000) Trust Score: 46.5% (+0.000) RECOVERY MODE,Analysis Rejection: NONE (0%) Sentiment: positive (+0.50) Curiosity: No Trust Issue: No


────────────────────────────────────────────────────────────────────────────────

